# title

In [123]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

- title
- url
- account name
- subscribers
- views
- date uploaded
- number of likes
- number of dislikes

In [172]:
def get_stats(youtube_url):
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    def km_cleaner(value):
        if "subscribers" in value:
            if 'K' in value:
                value = float(re.sub('K subscribers', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M subscribers', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
        elif "views" in value:
            if 'K' in value:
                value = float(re.sub('K views', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M views', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
        else:
            if 'K' in value:
                value = float(re.sub('K', '', value))
                value *= 1000
                return int(value)
            elif 'M' in value:
                value = float(re.sub('M', '', value))
                value *= 1000000
                return int(value)
            else:
                return int(value)
            
    title_value = soup.find('h1').text
    
    account_value = soup.find('yt-formatted-string', class_='ytd-channel-name').text
    
    subs_value = km_cleaner(soup.find('yt-formatted-string', id='owner-sub-count').text)
    
    view_count = km_cleaner(soup.find('span', class_='short-view-count').text)
    
    def date_uploaded(value):
        value = re.sub('•', '', value)
        if 'ago' in value:
            value = int(re.search(r'\d+', value).group())
            uploaded = (datetime.now() - timedelta(hours=value))
            return datetime.strftime(uploaded, '%b %-d, %Y')
        else:
            return value
    
    date_string = soup.find('div', id='date').text
    date_value = date_uploaded(date_string)
    
    likes_value = km_cleaner(soup.find_all('yt-formatted-string', class_='ytd-toggle-button-renderer')[0].text)
    dislikes_value = km_cleaner(soup.find_all('yt-formatted-string', class_='ytd-toggle-button-renderer')[1].text)

    
    stats = {'date_uploaded': date_value,
             'account': account_value,
             'video_title': title_value,
             'views': view_count,
             'subscribers': subs_value,
             'likes': likes_value,
             'dislikes': dislikes_value,
             'url': youtube_url
            }
    
    return stats 

In [177]:
def get_video_urls(youtube_playlist_url):
    #webdriver setup & html scrape
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_playlist_url)
    soup = BeautifulSoup(driver.page_source)

    #isolate the video stats
    video_stats = soup.find_all('yt-formatted-string')[5].text
    #use regex to isolate video number
    no_of_videos = re.sub(',', '', video_stats)
    no_of_videos = re.sub(' videos', '', video_stats)
    #number of scroll based on 100 thumbnails per scroll
    no_of_scrolls = int(int(no_of_videos) / 100 + 1)

    #scroll page for every 100 videos
    for i in range(no_of_scrolls):
        driver.execute_script("window.scrollBy(0, 12000);")
        time.sleep(3)

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    #extract youtube playlist urls
    matches = soup.find_all('a', class_='yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    youtube_urls = []
    for url in matches:
        string = url.get('href')
        substring = re.search('\/watch\?v=([^&]+)', string).group()
        youtube_urls.append('http://www.youtube.com' + substring)    
    return youtube_urls

In [173]:
def create_export(url):
    all_urls = get_video_urls(url)
    all_stats = []
    for video in all_urls:
        all_stats.append(get_stats(video))
    return all_stats

In [184]:
gerard = create_export('https://www.youtube.com/playlist?list=PL7qtZGedQPaeTqLFsqvY7h0ihZ4mH3XkH')

In [185]:
import pandas as pd

In [186]:
df = pd.DataFrame(gerard)

In [188]:
df

,account,video_title,date_uploaded,views,subscribers,likes,dislikes,url
0,Games Done Quick,Sonic CD by Rafalmir in 26:09 - Sonic and the ...,"Apr 29, 2020",17000,825000,396,11,http://www.youtube.com/watch?v=cU0JJL2nYY4
1,KQED,BBQ Pork and Side Dishes | Yan Can Cook | KQED,"Apr 27, 2020",6900,80100,40,2,http://www.youtube.com/watch?v=AVRk41nN2dg
2,WWE,Becky Lynch learns she can't compete at Surviv...,"Nov 16, 2018",1800000,59400000,33000,1200,http://www.youtube.com/watch?v=d-P6lpV_Ec4
3,Hats Off Entertainment,Jim Varney | The Original Viral Star | A Docu-...,"Apr 9, 2020",110000,8039,3600,31,http://www.youtube.com/watch?v=VNOpADr1vKs
4,Ganymede Cafe,fUNKY CityPop シティポップ JPOP 80s,"Jul 7, 2018",165000,21200,5100,100,http://www.youtube.com/watch?v=-_cmWZsJ5Ok
5,Rocco Botte,The TryMaster Tries New Stuff At Home - Quaran...,"May 2, 2020",12000,66700,1600,5,http://www.youtube.com/watch?v=E1TlI-_0WE0
6,SAM THE COOKING GUY,The Best Homemade McRib Sliders | SAM THE COOK...,"Jun 3, 2019",519000,1790000,10000,237,http://www.youtube.com/watch?v=VqL04h_6mkg
7,Saturday Night Live,Outside the Lines - SNL,"Sep 23, 2013",6200000,10100000,62000,1400,http://www.youtube.com/watch?v=Xez7kGgEg-U
